In [2]:
!pip3 install ../../../../../Downloads/torch_scatter-2.1.2-cp311-cp311-macosx_10_9_universal2.whl

Defaulting to user installation because normal site-packages is not writeable
ERROR: torch_scatter-2.1.2-cp311-cp311-macosx_10_9_universal2.whl is not a supported wheel on this platform.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os.path as osp
import torch
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

In [2]:
df_ori = pd.read_csv("tst.csv").drop(columns=["amount"])

In [3]:
df_ori

,company_id,investor_id
0,4708640,4708666
1,5310348,5310352
2,5835331,1468298
3,6339545,2901375
4,7140015,7140222
...,...,...
99995,71211319,12986379
99996,121778646,121778647
99997,36615212,4773902
99998,78441325,123692


In [4]:
## 涉及到的所有的节点：
node_order = sorted(
    list(
        set(
            df_ori.company_id.to_list() + df_ori.investor_id.to_list()
        )
    )
)

In [5]:
## 节点转为映射值：
mapping = {
    ci: idx for idx, ci in enumerate(node_order)
}
for col in ["investor_id", "company_id"]:
    df_ori[col] = df_ori[col].map(mapping)

In [6]:
from torch_geometric.data import Data

data = Data(
    num_nodes = len(mapping),
    edge_index=torch.tensor(
        df_ori.T.to_numpy(), 
        dtype = torch.long
    )
)

In [7]:
n_nodes = data.num_nodes
hidden_channels = 64

In [8]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,    
)
train_data, val_data, test_data = transform(data)

In [9]:
from torch_geometric.loader import LinkNeighborLoader
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[
        -1,# 10, 
        -1, # 5
    ],
    neg_sampling_ratio=2.0,
    batch_size=128,
    shuffle=True,
)

In [10]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, 128)
        self.conv2 = SAGEConv(128, out_channels)
    def forward(self, x, edge_index) :
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x_from, x_to,):
        return (x_from * x_to).sum(dim=-1)

class Model(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.emb = torch.nn.Embedding(n_nodes, in_channels)
        self.gnn = GNN(in_channels, out_channels)
        self.classifier = Classifier()
    def forward(self, data):
        x_out = self.gnn(
            self.emb(data.n_id), 
            data.edge_label_index
        )
        pred = self.classifier(
            x_out[data.edge_label_index[0]], ## 边的起始点。
            x_out[data.edge_label_index[-1]] ## 边的终结点。
        )
        return pred
        
model = Model(in_channels=hidden_channels, out_channels=64)

weights = model.emb.weight.detach().numpy()
# pd.DataFrame(weights, columns = [f"col_{i}" for i in range(weights.shape[1])])

In [33]:
import tqdm
import torch.nn.functional as F
device = torch.device("mps")#('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):        
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data.edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    
#     break
    
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")
    

Device: 'mps'


  0%|                                                   | 0/438 [00:00<?, ?it/s]


ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

In [17]:
!pip3 install pyg_lib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pyg_lib (from versions: none)
ERROR: No matching distribution found for pyg_lib
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [28]:
import torch_geometric

In [29]:
torch_geometric.__version__

'2.5.2'

In [36]:
!pip3 install pyg-lib -f https://data.pyg.org/whl/torch-2.2.2+cpu.html


Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-2.2.2+cpu.html
ERROR: Could not find a version that satisfies the requirement pyg-lib (from versions: none)
ERROR: No matching distribution found for pyg-lib
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [47]:
import pip3._internal

ModuleNotFoundError: No module named 'pip3'

In [44]:
pip._internal#.pep425tags.get_supported()

<module 'pip._internal' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages/pip/_internal/__init__.py'>

In [46]:
pip._internal.Optional

typing.Optional

In [ ]:
pip._internal.pep425tags

In [39]:
!pwd

/Users/minkexiu/Documents/GitHub/ML_Tryout/Graph_Series/20240304_quanweidu_gnn_rnd2


In [48]:
!pip3 install ../../../../../Downloads/torch_scatter-2.1.2-cp39-cp39-macosx_11_0_x86_64.whl

Defaulting to user installation because normal site-packages is not writeable
ERROR: torch_scatter-2.1.2+pt22cpu-cp39-cp39-linux_x86_64.whl is not a supported wheel on this platform.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [26]:
!pip3 install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-2.2.0+cpu.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  ERROR: Command errored out with exit status 1:
   command: /Library/Developer/CommandLineTools/usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/s1/1jpfx0m52rj4k7cgqkh7g3q40000gn/T/pip-install-ol7qgsi2/torch-scatter_ba4edb66c6a5469d9bb6c0481000d351/setup.py'"'"'; __file__='"'"'/private/var/folders/s1/1jpfx0m52rj4k7cgqkh7g3q40000gn/T/pip-install-ol7qgsi2/torch-scatter_ba4edb66c6a5469d9bb6c0481000d351/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code